In [53]:
import json
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
import random
import numpy as np
import pandas as pd
from collections import Counter, defaultdict

In [54]:
annotation = "/opt/ml/input/data/train_all.json"

with open(annotation) as f: 
    data = json.load(f)

In [ ]:
data.keys()

```python
dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])
```

In [ ]:
data["categories"] # background에 id가 부여되어 있지 않으며 annotations에도 background는 mask되어 있지 않음.

```python
[{'id': 1, 'name': 'General trash', 'supercategory': 'General trash'},
 {'id': 2, 'name': 'Paper', 'supercategory': 'Paper'},
 {'id': 3, 'name': 'Paper pack', 'supercategory': 'Paper pack'},
 {'id': 4, 'name': 'Metal', 'supercategory': 'Metal'},
 {'id': 5, 'name': 'Glass', 'supercategory': 'Glass'},
 {'id': 6, 'name': 'Plastic', 'supercategory': 'Plastic'},
 {'id': 7, 'name': 'Styrofoam', 'supercategory': 'Styrofoam'},
 {'id': 8, 'name': 'Plastic bag', 'supercategory': 'Plastic bag'},
 {'id': 9, 'name': 'Battery', 'supercategory': 'Battery'},
 {'id': 10, 'name': 'Clothing', 'supercategory': 'Clothing'}]
```

In [ ]:
data["images"] # [{}, {}, {}, {}] data["images"]["id"]

```python
[{'license': 0,
  'url': None,
  'file_name': 'batch_01_vt/0002.jpg',
  'height': 512,
  'width': 512,
  'date_captured': None,
  'id': 0},
 {'license': 0,
  'url': None,
  'file_name': 'batch_01_vt/0003.jpg',
  'height': 512,
  'width': 512,
  'date_captured': None,
  'id': 1},
  ...]
```

In [ ]:
# 각 annotation은 object기준 으로 리스트에 담겨 있으며 리스트의 element는 아래와 같음
data["annotations"][1] 

```python
Output exceeds the size limit. Open the full output data in a text editor
{'id': 1,
 'image_id': 0,
 'category_id': 8,
 'segmentation': [[482,
   248,
   481,
   248,
   480,
...
   482,
   248]],
 'area': 3313.5,
 'bbox': [413.2, 196.1, 72.6, 52.7],
 'iscrowd': 0}
```

In [ ]:
# 같은 카테고리라도 영역이 아래처럼 instance별로 영역이 나누어져서 표기됨
[[len(i) for i in ann["segmentation"]] for ann in data['annotations']] 

```python
Output exceeds the size limit. Open the full output data in a text editor
[[410, 618, 454],
 [542],
 [710],
 [1226],
...
 [1330],
 [986],
 [1150],
 [894],
 ...]
```

In [ ]:
## pixel별 분류는 보류

# data["annotations"]는 리스트 [ {}, {}, {}]
# data["annotaitons"] 리스트에서 image_id와 category id를 가져옴
var = [(ann['image_id'], ann['category_id'], sum([len(i) for i in ann["segmentation"]])) for ann in data['annotations']] 

# annotation 길이 만큼의 1로 이루어진 X를 만듬 [1,1,1,.....1,1]
X = np.ones((len(data['annotations']),1)) 

# annotation의 image_id
groups = np.array([v[0] for v in var]) # 각 annotation 마다 image_id가 담긴 group

# annotations의 category_id(annotation은 이것 기준으로 나눴음)
y1 = np.array([v[1] for v in var]) # 각 annotation마다 category가 담긴 y1
y2 = np.array([v[2] for v in var]) # 각 annotation마다 category별로 pixel 개수가 담긴 y2
df = pd.DataFrame(y2)

bins = np.linspace(df.min(), df.max(), 4)

# 픽셀 분류기준
print("threshold 0:", np.percentile(y2, 25))
print("threshold 1:", np.percentile(y2, 50))
print("threshold 2:", np.percentile(y2, 70))


```python
threshold 0: 376.0
threshold 1: 766.0
threshold 2: 1234.0
```

In [ ]:
# 5 fold
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

"""
split(X, y=None, groups=None)

Xarray-like of shape (n_samples, n_features)
Training data, where n_samples is the number of samples and n_features is the number of features.

yarray-like of shape (n_samples,), default=None
The target variable for supervised learning problems.

groupsarray-like of shape (n_samples,), default=None
Group labels for the samples used while splitting the dataset into train/test set.

"""

# annotation의 train, valid idx를 가져옴
# Group의 비율을 유지하고 + y의 비율을 유지하면서 쪼개서 인덱스를 리턴해라
# X는 의미없음 data를 넣어도 되는데 어자피 x의 index를 리턴해서 그냥 data와 같은 길이의 X를 넣어도 무관함
fold_idx = dict()
for idx, (train_idx, val_idx) in enumerate(cv.split(X, y1, groups)): # x 
    print("Fold:", idx)    
    print("    TRAIN:", len(groups[train_idx]))
    print("    TEST:", len(groups[val_idx]))
    fold_idx[f"train{idx}"] = train_idx
    fold_idx[f"val{idx}"] = val_idx

```python
Fold: 0
    TRAIN: 20923
    TEST: 5317
Fold: 1
    TRAIN: 20921
    TEST: 5319
Fold: 2
    TRAIN: 21015
    TEST: 5225
Fold: 3
    TRAIN: 21138
    TEST: 5102
Fold: 4
    TRAIN: 20963
    TEST: 5277
```

In [62]:
# 정상 분류 확인
# 아래 dataframe 결과 background는 annotation에 없음

def get_distribution(y):
    y_distr = Counter(y)
    y_vals_sum = sum(y_distr.values())

    return [f'{y_distr[i]/y_vals_sum:.2%}' for i in range(np.max(y) +1)]

distrs = [get_distribution(y1)]
index = ['training set']

for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X,y1, groups)):
    train_y, val_y = y1[train_idx], y1[val_idx]
    train_gr, val_gr = groups[train_idx], groups[val_idx]
    assert len(set(train_gr) & set(val_gr)) == 0 

    distrs.append(get_distribution(train_y))
    distrs.append(get_distribution(val_y))
    index.append(f'train - fold{fold_ind}')
    index.append(f'val - fold{fold_ind}')

categories = ["Background", "General trash", "Paper", 
              "Paper pack", "Metal", "Glass", "Plastic", 
              "Styrofoam", "Plastic bag", "Battery", "Clothing"]
pd.DataFrame(distrs, index=index, columns=categories)

,Background,General trash,Paper,Paper pack,Metal,Glass,Plastic,Styrofoam,Plastic bag,Battery,Clothing
training set,0.00%,10.60%,35.48%,2.51%,2.14%,2.32%,11.78%,5.12%,29.13%,0.24%,0.67%
train - fold0,0.00%,10.81%,35.57%,2.49%,2.05%,2.26%,11.75%,5.12%,29.04%,0.24%,0.68%
val - fold0,0.00%,9.80%,35.15%,2.60%,2.52%,2.60%,11.87%,5.12%,29.47%,0.23%,0.66%
train - fold1,0.00%,10.54%,35.36%,2.41%,2.24%,2.37%,11.73%,5.32%,29.09%,0.23%,0.71%
val - fold1,0.00%,10.83%,35.98%,2.90%,1.77%,2.16%,11.94%,4.31%,29.29%,0.28%,0.55%
train - fold2,0.00%,10.56%,35.28%,2.57%,2.00%,2.31%,11.70%,5.17%,29.52%,0.20%,0.69%
val - fold2,0.00%,10.78%,36.29%,2.26%,2.72%,2.39%,12.08%,4.92%,27.54%,0.40%,0.63%
train - fold3,0.00%,10.70%,35.67%,2.65%,2.28%,2.29%,11.69%,4.89%,28.94%,0.26%,0.63%
val - fold3,0.00%,10.19%,34.71%,1.92%,1.55%,2.47%,12.15%,6.06%,29.91%,0.18%,0.86%
train - fold4,0.00%,10.40%,35.54%,2.42%,2.14%,2.40%,12.01%,5.09%,29.05%,0.27%,0.67%


In [63]:
train_indices = []
val_indices = []
for i in range(0,len(fold_idx)//2):
    train_idx = groups[fold_idx[f"train{i}"]] # fold i에 담긴 image id
    val_idx = groups[fold_idx[f"val{i}"]] # fold i에 담긴 valid image id
    train_indices.append(train_idx)
    val_indices.append(val_idx)


In [ ]:
train_jsons = []

for ord, train_idx in enumerate(train_indices):
    # print(train_idx) # 이미지 ids, ex) like [   0    0    0 ... 3270 3270 3271]

    # 아래 3개는 full_train과 똑같습니다.
    fold_json = dict()

    fold_json["info"] = data["info"]
    fold_json["licenses"] = data["licenses"]
    
    
    fold_img_dict = []
    fold_img_id_list = [] # 편의상 선언함
    fold_img_annot = []
    
    # 모든 이미지를 돌면서
    for img_dict in data["images"]: # data["images"]["id"]
        # train_idx에 id 있는 이미지면 img_dict를 뽑아옵니다
        if img_dict["id"] in train_idx:
            fold_img_dict.append(img_dict)
            fold_img_id_list.append(img_dict["id"])
    
    # 모든 annot를 돌면서 돌면서
    for annot in data["annotations"]:
        # annot가 소속된 이미지 id가 
        # 지금 폴드에 있는 이미지면
        if annot["image_id"] in fold_img_id_list: 
            # annot를 담습니다
            fold_img_annot.append(annot) 
        
    fold_json["images"] = fold_img_dict
    fold_json["categories"] = data["categories"]
    fold_json["annotations"] = fold_img_annot
    
    train_jsons.append(fold_json)

    print(f"fold{ord}에는 총 {len(fold_img_dict)}개 train 이미지가 존재합니다")
    print(f"fold{ord} 작업이 종료되었습니다")
    print()

```python
fold0에는 총 2618개 train 이미지가 존재합니다
fold0 작업이 종료되었습니다

fold1에는 총 2615개 train 이미지가 존재합니다
fold1 작업이 종료되었습니다

fold2에는 총 2614개 train 이미지가 존재합니다
fold2 작업이 종료되었습니다

fold3에는 총 2624개 train 이미지가 존재합니다
fold3 작업이 종료되었습니다

fold4에는 총 2613개 train 이미지가 존재합니다
fold4 작업이 종료되었습니다
```

In [ ]:
val_jsons = []

for ord, val_idx in enumerate(val_indices):

    # 아래 3개는 full_train과 똑같습니다.
    fold_json = dict()

    fold_json["info"] = data["info"]
    fold_json["licenses"] = data["licenses"]
    fold_json["categories"] = data["categories"]
    
    fold_img_dict = []
    fold_img_id_list = [] # 편의상 선언함
    fold_img_annot = []
    
    # 모든 이미지를 돌면서
    for img_dict in data["images"]: # data["images"]["id"]
        # val_idx id 있는 이미지면 img_dict를 뽑아옵니다
        if img_dict["id"] in val_idx:
            fold_img_dict.append(img_dict)
            fold_img_id_list.append(img_dict["id"])
    
    # 모든 annot를 돌면서 돌면서
    for annot in data["annotations"]:
        # annot가 소속된 이미지 id가 
        # 지금 폴드에 있는 이미지면
        if annot["image_id"] in fold_img_id_list: 
            # annot를 담습니다
            fold_img_annot.append(annot) 
        
    fold_json["images"] = fold_img_dict
    fold_json["annotations"] = fold_img_annot
    
    val_jsons.append(fold_json)

    print(f"fold{ord}에는 총 {len(fold_img_dict)}개 valid 이미지가 존재합니다")
    print(f"fold{ord} 작업이 종료되었습니다")
    print()

```python
fold0에는 총 653개 valid 이미지가 존재합니다
fold0 작업이 종료되었습니다

fold1에는 총 656개 valid 이미지가 존재합니다
fold1 작업이 종료되었습니다

fold2에는 총 657개 valid 이미지가 존재합니다
fold2 작업이 종료되었습니다

fold3에는 총 647개 valid 이미지가 존재합니다
fold3 작업이 종료되었습니다

fold4에는 총 658개 valid 이미지가 존재합니다
fold4 작업이 종료되었습니다
```

In [66]:
for ord, (train_dict, val_dict) in enumerate(zip(train_jsons, val_jsons)):
    with open(f'train_fold_{ord}.json', 'w') as f:
        json.dump(train_dict, f, indent=4)
    with open(f'val_fold_{ord}.json', 'w') as f:
        json.dump(val_dict, f, indent=4)